In [ ]:
# pickle 파일 읽기
import pickle
import gzip

with gzip.open('hs.pickle', 'rb') as f:
    data_hs = pickle.load(f)

data_hs

In [ ]:
# pickle 파일 읽기
import pickle
import gzip

with gzip.open('khs.pickle', 'rb') as f:
    data_khs = pickle.load(f)

data_khs

In [ ]:
# pickle 파일 읽기
import pickle
import gzip

with gzip.open('hs_val.pickle', 'rb') as f:
    data_hs_val = pickle.load(f)

data_hs_val

In [ ]:
# pickle 파일 읽기
import pickle
import gzip

with gzip.open('kold.pickle', 'rb') as f:
    data_kold = pickle.load(f)

data_kold

In [6]:
import pandas as pd

X_data = pd.concat([data_hs['onehot_vector'], data_khs['onehot_vector'], data_hs_val['onehot_vector'], data_kold['onehot_vector']],ignore_index=True)
X_data

0        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...
1        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...
2        [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
3        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
4        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...
                               ...                        
57862    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
57863    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...
57864    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,...
57865    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...
57866    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...
Name: onehot_vector, Length: 57867, dtype: object

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim

In [7]:
tensor_list = [torch.tensor(array) for array in X_data.values]
X_data = torch.stack([torch.unsqueeze(tensor, dim=0) for tensor in tensor_list], dim=0)
X_data.shape

torch.Size([57867, 1, 200, 127])

In [9]:
import pandas as pd

y_data = pd.concat([data_hs['clean'], data_khs['clean'], data_hs_val['clean'], data_kold['clean']],ignore_index=True)
y_data

0        1
1        0
2        1
3        1
4        0
        ..
57862    0
57863    0
57864    0
57865    1
57866    1
Name: clean, Length: 57867, dtype: object

In [10]:
y_data = torch.tensor(y_data)
y_data = torch.unsqueeze(y_data, dim=1)
y_data.shape

torch.Size([57867, 1])

In [3]:
from torch.utils.data import Dataset

class YourDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __len__(self):
        return len(self.X_data)
    
    def __getitem__(self, index):
        X = self.X_data[index]
        y = self.y_data[index]
        
        return X, y


In [4]:
from sklearn.model_selection import train_test_split

# 데이터를 훈련 세트와 검증 세트로 나눔
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

NameError: name 'X_data' is not defined

In [104]:
batch_size = 16
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=6, padding=(193, 708))
        self.pool = nn.MaxPool2d(kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=9, padding=(182,517))
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=12, padding=(170,518))
        self.fc1 = nn.Linear(batch_size*170*512, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 1)
        self.dropout = nn.Dropout(p=0.5)  # Dropout 추가

    def forward(self, x):
        print(x.size())
        x = self.pool(torch.relu(self.conv1(x)))
        print(1, x.size())
        x = self.pool(torch.relu(self.conv2(x)))
        print(2, x.size())
        x = self.pool(torch.relu(self.conv3(x)))
        print(3, x.size())
        x = torch.flatten(x)  # Flatten
        print(4, x.size())
        x = torch.relu(self.fc1(x))
        print(5, x.size())
        x = self.dropout(x)  # Dropout
        print(6, x.size())
        x = torch.relu(self.fc2(x))
        print(7, x.size())
        x = self.dropout(x)  # Dropout
        print(8, x.size())
        x = torch.sigmoid(self.fc3(x))  # 이진 분류를 위한 sigmoid 함수 사용
        print(9, x.size())
        return x

# 모델 인스턴스 생성
model = CNN()

# Loss function 및 Optimizer 정의
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Learning rate scheduler 정의
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)


In [2]:
from torch.utils.data import DataLoader

dataset = YourDataset(X_train, y_train)  # YourDataset은 실제 데이터셋을 로드하고 전처리하는 클래스입니다.
# batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)  # batch_size는 한 번에 모델에 전달되는 데이터의 수입니다.

# 데이터셋 객체를 생성
val_dataset = YourDataset(X_val, y_val)  # X_val은 검증 데이터의 입력, y_val은 검증 데이터의 타겟
# DataLoader로 데이터셋을 배치 단위로 로드
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)  # batch_size는 한 번에 모델에 전달되는 데이터의 수입니다.

# 모델 학습
num_epochs = 100
for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    running_loss = 0.0
    for inputs, labels in train_loader:  # DataLoader에서 배치 단위로 데이터 로드
        print(inputs.size(), labels.size())
        optimizer.zero_grad()  # gradient 초기화
        inputs = inputs.float()
        outputs = model(inputs).float()  # 모델 예측
        print(outputs)
        outputs = outputs.expand(*(batch_size, 1))
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 옵티마이저 업데이트
        running_loss += loss.item() * inputs.size(0)
    scheduler.step()  # Learning rate 스케줄링

    # 현재 epoch의 평균 손실 출력
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# 검증 데이터를 사용하여 모델 검증
model.eval()  # 모델을 평가 모드로 설정
correct = 0
total = 0
with torch.no_grad():  # 평가 과정에서는 gradient를 계산하지 않음
    for inputs, labels in val_loader:  # 검증 데이터 로드
        outputs = model(inputs)  # 모델 예측
        predicted = (outputs >= 0.5).float()  # 확률을 이진 분류로 변환
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 정확도 출력
accuracy = correct / total
print(f"Validation Accuracy: {accuracy:.2%}")


NameError: name 'YourDataset' is not defined

In [118]:
import torch
import torch.nn as nn

# 입력 데이터
inputs = torch.randn(1, 1, 200, 127)  # batch_size x channels x height x width

# Convolutional layer 정의
conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=6, padding=(193,708))  # padding=(height_pad, width_pad)

# Convolution 연산
output_conv1 = conv1(inputs)
print("Output after convolution:", output_conv1.size())

# MaxPooling 연산
pool = nn.MaxPool2d(kernel_size=3)
output_pool = pool(output_conv1)
print("Output after MaxPooling:", output_pool.size())

# Convolutiona2 layer 정의
conv2 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=9, padding=(182,517))  # padding=(height_pad, width_pad)

# Convolution 연산
output_conv2 = conv2(output_pool)
print("Output after convolution:", output_conv2.size())

# MaxPooling 연산
pool = nn.MaxPool2d(kernel_size=3)
output_pool = pool(output_conv2)
print("Output after MaxPooling:", output_pool.size())

# Convolutiona3 layer 정의
conv3 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=12, padding=(170,518))  # padding=(height_pad, width_pad)

# Convolution 연산
output_conv3 = conv3(output_pool)
print("Output after convolution:", output_conv3.size())

# MaxPooling 연산
pool = nn.MaxPool2d(kernel_size=3)
output_pool = pool(output_conv3)
print("Output after MaxPooling:", output_pool.size())

output_pool = torch.flatten(output_pool,1)
fc1 = nn.Linear(170*512, 1024)
output = fc1(output_pool)
print(output.size())

fc2 = nn.Linear(1024, 1024)
output = fc2(output)
print(output.size())

fc3 = nn.Linear(1024, 1)
output = fc3(output)
print(output.size())

print(output.expand(*(16,1)).size())

Output after convolution: torch.Size([1, 1, 581, 1538])
Output after MaxPooling: torch.Size([1, 1, 193, 512])
Output after convolution: torch.Size([1, 1, 549, 1538])
Output after MaxPooling: torch.Size([1, 1, 183, 512])
Output after convolution: torch.Size([1, 1, 512, 1537])
Output after MaxPooling: torch.Size([1, 1, 170, 512])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1])
torch.Size([16, 1])
